In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Load DFs

In [ ]:
squeeth_bt_df = pd.read_csv('strategy_dfs/squeeth_backtest_df.csv', index_col=0)
deribit_bt_df = pd.read_csv('strategy_dfs/deribit_backtest_df.csv', index_col=0)

# Plot IV Comparison

In [ ]:
def plot_iv_comparison(squeeth_bt_df, deribit_bt_df):
    squeeth_bt_df.index = pd.to_datetime(squeeth_bt_df.index, unit='ns')
    deribit_bt_df['freq_timestamp'] = pd.to_datetime(deribit_bt_df['freq_timestamp'], unit='ns')
    grouped_df =  deribit_bt_df.groupby('freq_timestamp')
    
    iv_comparison_df = pd.DataFrame({
        '17.5-day ATM IV': grouped_df['smile_avg_vol_17.5_days'].last().ewm(halflife=1).mean(),
        '17.5-day DVOL': grouped_df['dvol_17.5_days'].last().ewm(halflife=1).mean(),
        '17.5-day Option IV': grouped_df['2perp_vol_17.5_days'].last().ewm(halflife=1).mean(),
        'Squeeth IV': squeeth_bt_df['osqth_iv'].ewm(halflife=3).mean() 
    })
    
    iv_comparison_df.dropna().plot(figsize=(20, 10), grid=True)
    plt.ylabel('Vol', fontsize=14)
    plt.legend(loc=1, prop={'size': 14})
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
    return iv_comparison_df

In [ ]:
ivs_df = plot_iv_comparison(squeeth_bt_df, deribit_bt_df)

# Power-2 Perp vs Power-2 Future

In [ ]:
def power2_perp_price(S, N, sigma, T):
    carry_term = (N + 1)*np.exp(-T*(sigma**2/N)) - N
    return S**2 / carry_term

def power2_perp_iv(S, N=420, sigma=0.9, funding_period_days=17.5):
    T = funding_period_days / 365.
    D = power2_perp_price(S, N, sigma, T)
    return np.sqrt(N*np.log((D*(N + 1))/(D*N + S**2)))*np.sqrt(365. / funding_period_days)

def power2_perp_approx_iv(S, N=420, sigma=0.9, days_to_expiry=17.5):
    T = days_to_expiry / 365.
    D = power2_perp_price(S, N, sigma, T)
    return np.sqrt(np.log(D/S**2))*np.sqrt(365./days_to_expiry)

def power2_future_price(S, sigma, T):
    carry_term = np.exp(T*sigma**2)
    return S**2 * carry_term

def power2_future_iv(S, sigma=0.9, days_to_expiry=17.5):
    T = days_to_expiry / 365.
    cost = power2_future_price(S, sigma, T)
    return np.sqrt(np.log(cost/S**2))*np.sqrt(365./days_to_expiry)

In [ ]:
def compare_perp_future_prices(
    underlying_price=2500, 
    num_funding_periods=420, 
    funding_period_days=17.5,
    sigma_range=np.arange(0.5, 2.5, 0.5)
):
    T = funding_period_days / 365.
    
    perp_prices, future_prices = [], []
    for sigma in sigma_range:
        perp_prices.append(power2_perp_price(underlying_price, num_funding_periods, sigma, T))
        future_prices.append(power2_future_price(underlying_price, sigma, T))
    
    perp_prices = np.asarray(perp_prices)
    future_prices = np.asarray(future_prices)
    perc_diff = np.log(perp_prices / future_prices)
    
    plt.plot(sigma_range, perc_diff)
    plt.ylabel('% Price Diff')
    plt.xlabel('Vol')
    plt.grid(True)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.show()
    
    plt.plot(sigma_range, perp_prices, label='Power-2 Perp, Funding Period = {} days'.format(funding_period_days))
    plt.plot(sigma_range, future_prices, label='Power-2 Future, Expiration = {} days'.format(funding_period_days))
    plt.ylabel('Price')
    plt.xlabel('Vol')
    plt.legend()
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.show()
    
    return {'perp_prices': perp_prices, 'future_prices': future_prices}

In [ ]:
results = compare_perp_future_prices()

In [ ]:
def compare_perp_future_ivs(
    underlying_price=2500, 
    num_funding_periods=420, 
    funding_period_days=17.5,
    sigma_range=np.arange(0.5, 2.5, 0.5)
):
    
    perp_ivs, approx_perp_ivs = [], []
    for sigma in sigma_range:
        perp_ivs.append(power2_perp_iv(underlying_price, num_funding_periods, sigma, funding_period_days))
        approx_perp_ivs.append(power2_perp_approx_iv(underlying_price, num_funding_periods, sigma, funding_period_days))
    
    perp_ivs = np.asarray(perp_ivs)
    approx_perp_ivs = np.asarray(approx_perp_ivs)
    perc_diff = np.log(approx_perp_ivs / perp_ivs)
    
    plt.plot(sigma_range, perc_diff)
    plt.ylabel('% IV Diff')
    plt.xlabel('Vol')
    plt.grid(True)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.show()

    plt.plot(sigma_range, perp_ivs, label='Power-2 Perp, Funding Period = {} days'.format(funding_period_days))
    plt.plot(sigma_range, approx_perp_ivs, label='Approx Power-2 Perp')
    plt.ylabel('IV')
    plt.xlabel('Vol')
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return {'perp_ivs': perp_ivs, 'approx_perp_ivs': approx_perp_ivs}

In [ ]:
results = compare_perp_future_ivs()

# Power-2 Future vs Options

In [ ]:
def option_price(S, K, sigma, T, option_type='call'):
    d1 = (np.log(S/K) + T*(sigma**2/2)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        return S*norm.cdf(d1) - K*norm.cdf(d2)
    elif option_type == 'put':
        return K*norm.cdf(-d2) - S*norm.cdf(-d1)
    
def option_rep_port_price(S, dK, sigma, T, max_K_mul=4):
    K_0 = S
    min_K = 0
    max_K = max_K_mul*S
    
    put_strikes = np.arange(min_K, K_0, dK)
    call_strikes = np.arange(K_0, max_K + dK, dK)

    put_cost = 2*dK*np.sum(np.asarray([option_price(S, K, sigma, T, 'put') for K in put_strikes]))
    call_cost = 2*dK*np.sum(np.asarray([option_price(S, K, sigma, T, 'call') for K in call_strikes]))
        
    return K_0**2 + 2*K_0*(S - K_0) + call_cost + put_cost
    
def option_rep_port_iv(S, dK=1, sigma=0.9, days_to_expiry=17.5):
    T = days_to_expiry / 365.
    cost = option_rep_port_price(S, dK, sigma, T)
    return np.sqrt(np.log(cost / S**2))*np.sqrt(365. / days_to_expiry)

In [ ]:
def compare_options_port_future(
    underlying_price=2500,
    dK=1,
    sigma_range=np.arange(0.5, 2.5, 0.5),
    days_to_expiry=17.5,
):
    option_ivs, future_ivs = [], []
    for sigma in sigma_range:
        option_ivs.append(option_rep_port_iv(underlying_price, dK, sigma, days_to_expiry))
        future_ivs.append(power2_future_iv(underlying_price, sigma, days_to_expiry))
    
    option_ivs = np.asarray(option_ivs)
    future_ivs = np.asarray(future_ivs)

    option_prices, future_prices = [], []
    for sigma in sigma_range:
        option_prices.append(option_rep_port_price(underlying_price, dK, sigma, days_to_expiry / 365.))
        future_prices.append(power2_future_price(underlying_price, sigma, days_to_expiry / 365.))

    option_prices = np.asarray(option_prices)
    future_prices = np.asarray(future_prices)

    plt.plot(sigma_range, option_ivs, label='Option IV, Expiration = {} days'.format(days_to_expiry))
    plt.plot(sigma_range, future_ivs, label='Power-2 Future IV, Expiration = {} days'.format(days_to_expiry))
    plt.ylabel('IV')
    plt.xlabel('Vol')
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.legend()
    plt.grid(True)
    plt.show()
    
    plt.plot(sigma_range, option_prices, label='Option Price, Expiration = {} days'.format(days_to_expiry))
    plt.plot(sigma_range, future_prices, label='Power-2 Future Price, Expiration = {} days'.format(days_to_expiry))
    plt.ylabel('Price')
    plt.xlabel('Vol')
    plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.legend()
    plt.grid(True)
    plt.show()

    return {
        'option_ivs': option_ivs, 
        'future_ivs': future_ivs, 
        'option_prices': option_prices, 
        'future_prices': future_prices
    }

def compare_option_dK_IV(
    underlying_price=2500,
    dKs=[0.1, 1, 10, 25, 50, 100],
    sigma=0.9,
    days_to_expiry=17.5
):
    option_dK_ivs = [option_rep_port_iv(underlying_price, dK, sigma, days_to_expiry) for dK in dKs]
    option_dK_ivs = np.asarray(option_dK_ivs)
    
    plt.plot(dKs, option_dK_ivs)
    plt.xscale('log')
    plt.ylabel('IV')
    plt.xlabel('dK')
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.grid(True)
    plt.show()
    
    return {'option_dK_ivs': option_dK_ivs}

In [ ]:
results = compare_options_port_future()

In [ ]:
results = compare_option_dK_IV()

# Exact Squeeth Replicating Portfolio

In [ ]:
import seaborn as sns

def squeeth_rep_port_option_notional(
    num_funding_periods=35,
    funding_period_days=17.5,
    dk=100,
    S=2500,
    max_i=100
):
    min_K = 0
    max_K = 2*S
    N = num_funding_periods
    P = funding_period_days
        
    strikes = np.arange(min_K, max_K + dk, dk)
    expiries = np.asarray([(P/N)*i for i in range(1, max_i)])
    
    expiry_weights = np.asarray([(1/N)*(N/(N+1))**i for i in range(1, max_i)]).reshape(-1, 1)
    strike_weights = np.full(shape=(len(expiries), len(strikes)), fill_value=2*dk)
    option_weights = expiry_weights*strike_weights
    option_weights_df = pd.DataFrame(option_weights, index=expiries, columns=strikes)
    option_weights_plot_df = option_weights_df[option_weights_df.index <= 2*funding_period_days]

    hm = sns.heatmap(option_weights_plot_df, annot=False, linecolor='white', linewidths=0.05)
    hm.set_xlabel('Strike Price')
    hm.set_ylabel('Days to Expiry')
    hm.set_title('# of Options')
    
    return option_weights_df

In [ ]:
notional_weights = squeeth_rep_port_option_notional()